In [ ]:
!pip install datasets
!pip install tqdm
!pip install transformers
!pip install torch
!pip install dataclasses
!pip install evaluate
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset, Audio, DatasetDict
from tqdm import tqdm
from tqdm.notebook import tqdm
from transformers import WhisperFeatureExtractor, WhisperForConditionalGeneration, WhisperTokenizer, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
fleurs_dataset = load_dataset("cobrayyxx/FLEURS_INDO-ENG_Speech_Translation",trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/458M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/457M [00:00<?, ?B/s]

...

train-00004-of-00005.parquet:   0%|          | 0.00/462M [00:00<?, ?B/s]

validation-00000-of-00002.parquet:   0%|          | 0.00/275M [00:00<?, ?B/s]

validation-00001-of-00002.parquet:   0%|          | 0.00/281M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2848 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/713 [00:00<?, ? examples/s]

In [ ]:
fleurs_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'text_indo', 'text_en'],
        num_rows: 2848
    })
    validation: Dataset({
        features: ['id', 'audio', 'text_indo', 'text_en'],
        num_rows: 713
    })
})

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="en", task="translate")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
input_str = fleurs_dataset["train"]["text_en"][0]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Some scientists think Triceratops ate cycads, which are a type of plant that was common in the Cretaceous.
Decoded w/ special:    <|startoftranscript|><|en|><|translate|><|notimestamps|>Some scientists think Triceratops ate cycads, which are a type of plant that was common in the Cretaceous.<|endoftext|>
Decoded w/out special: Some scientists think Triceratops ate cycads, which are a type of plant that was common in the Cretaceous.
Are equal:             True


In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="en", task="translate")

In [ ]:
fleurs_dataset["train"][0]

{'id': 370,
 'audio': {'path': '6166977075470649137.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.78813934e-07,  4.09364700e-04,  4.82201576e-05]),
  'sampling_rate': 16000},
 'text_indo': 'Beberapa ilmuwan mengira Triceratop memakan palem sagu, yang merupakan jenis tumbuhan yang umum di zaman Kapur.',
 'text_en': 'Some scientists think Triceratops ate cycads, which are a type of plant that was common in the Cretaceous.'}

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    #compute log-Me1 input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text_en"]).input_ids
    return batch

In [ ]:
fleurs_dataset = fleurs_dataset.map(prepare_dataset, remove_columns=fleurs_dataset.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/2848 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/713 [00:00<?, ? examples/s]

In [ ]:
fleurs_dataset

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.generation_config.language = "en"
model.generation_config.task = "translate"

model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
metric = evaluate.load("wer")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-indo-eng",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=100,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=fleurs_dataset["train"],
    eval_dataset=fleurs_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-28-6290d18f94c9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=100, training_loss=2.548831787109375, metrics={'train_runtime': 476.6016, 'train_samples_per_second': 3.357, 'train_steps_per_second': 0.21, 'total_flos': 4.61736640512e+17, 'train_loss': 2.548831787109375, 'epoch': 0.5617977528089888})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/cobrayyxx/whisper-small-indo-eng/commit/094e9d933b1c4a9deb6133e00ce31062562ddf01', commit_message='End of training', commit_description='', oid='094e9d933b1c4a9deb6133e00ce31062562ddf01', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cobrayyxx/whisper-small-indo-eng', endpoint='https://huggingface.co', repo_type='model', repo_id='cobrayyxx/whisper-small-indo-eng'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import WhisperTokenizerFast

# Load your fine-tuned tokenizer
tokenizer = WhisperTokenizerFast.from_pretrained("openai/whisper-small", language="en", task="translate")

# Save the tokenizer locally
tokenizer.save_pretrained("whisper-small-indo-eng",legacy_format=False)

# Push the tokenizer to the Hugging Face Hub
tokenizer.push_to_hub("cobrayyxx/whisper-small-indo-eng")

CommitInfo(commit_url='https://huggingface.co/cobrayyxx/whisper-small-indo-eng/commit/c0c3955292872efd43b848e1264fafacbfcd6923', commit_message='Upload tokenizer', commit_description='', oid='c0c3955292872efd43b848e1264fafacbfcd6923', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cobrayyxx/whisper-small-indo-eng', endpoint='https://huggingface.co', repo_type='model', repo_id='cobrayyxx/whisper-small-indo-eng'), pr_revision=None, pr_num=None)